In [337]:
import pandas as pd
import numpy as np
import random
import ast


In [367]:
ngrams_value = 2
total_patients = 10
maximum_visits = 10
total_features = 7
train_percentage = 0.9
train_pid_remove=True
test_pid_remove=False

In [368]:
def create_dummy_df(num_patients, num_max_visits, num_features):
    random_patient_visits = [random.randint(4,num_max_visits) for _ in range(num_patients)]
    dummy_df = [[[random.randint(1,10) for _ in range(num_features)] for num_visits in range(random_patient_visits[patient_index])] for patient_index in range(num_patients)]
    for i,record in enumerate(dummy_df):
        for ind_rec in record:
            ind_rec[0] = i+1
    for num_patient,patient in enumerate(dummy_df):
        pad_needed = num_max_visits - random_patient_visits[num_patient]
        if(pad_needed !=0):
            zero_pad = [[0]*num_features for _ in range(pad_needed)]
            zero_pad.extend(list(patient))
            dummy_df[num_patient] = zero_pad
    dummy_df = np.asarray(dummy_df)
    return dummy_df
    

In [369]:
df = create_dummy_df(total_patients, maximum_visits, total_features)

In [370]:
def generate_ngrams(s, n):
    #Assume 0th index to be PID
    l = len(s)
    ngrams = []
    for rec_index in range(l-1,-1,-1):
        temp_list = [s[i] for i in range(rec_index,rec_index-n,-1)]
        for record in temp_list:
            if(record[0] == 0):
                return ngrams
        ngrams.append(temp_list)
#     ngrams = [[s[i] for i in range(j,j-n,-1) if sum(s[i])!=0] for j in range(l-1,-1,-1) if sum(s[j])!=0]
    return ngrams

In [371]:
def gen_ngrams(df, n = 2):
    total_patient_bigrams = []
    index_list = []
    for patient_index in range(df.shape[0]):
        patient_details = np.asarray(df[patient_index])
        if(len(patient_details)>=n):
            patient_ngrams = generate_ngrams(patient_details,n)
            total_patient_bigrams.extend(patient_ngrams)
            index_list.extend([patient_ngrams[0][0][0] for x in range(len(patient_ngrams))])
    num_total_patient_bigrams = len(total_patient_bigrams)
    return num_total_patient_bigrams,n,np.asarray(total_patient_bigrams),index_list



In [372]:

num_patient,n,ngrams,index_list = gen_ngrams(df,ngrams_value)
# print(ngrams)

In [373]:
#Train/Test Data Split 

from itertools import groupby
from itertools import permutations

def remove_pid(record):
    record = record.tolist()
    for i,encounter in enumerate(record):
        record[i] = encounter[1:]
    return record

def find_closest_sum(numbers, target, n):
        permlist = list(permutations(numbers, n))
        sumlist = [sum(l) for l in permlist]
        maxpos = 0
        for i in range(1, len(sumlist)):
            if abs(sumlist[i] - target) < abs(sumlist[maxpos]-target):
                 maxpos = i
        return permlist[maxpos]
                    
def split(index_list,ngrams,train_percentage = 0.5,train_pid_remove = True,test_pid_remove = True):
    freq_dict = {key:len(list(group)) for key, group in groupby(index_list)}
    target = int(len(index_list)*train_percentage)
    freq_values = list(freq_dict.values())
    prev_diff = maximum_visits
    best_index = []
    for num_train_users in range(1,total_patients+1):
        indexes = find_closest_sum(freq_values, target, num_train_users)
        s_index = sum(indexes)
        if(target - s_index < prev_diff):
            prev_diff = target - s_index
            best_index = indexes
            if(prev_diff == 0):
                break
    train_pid = []
    for val in best_index:
        ind = freq_values.index(val)
        freq_values[ind] = -1
        train_pid.append(ind+1)
    test_pid = [x for x in range(1,total_patients+1) if x not in train_pid]
    train_data = []
    test_data = []
    for record in ngrams:
        if(record[0][0] in train_pid):
            if(train_pid_remove):
                record = remove_pid(record)
            train_data.append(record)
        else:
            if(test_pid_remove):
                record = remove_pid(record)
            test_data.append(record)
    return np.asarray(train_data),np.asarray(test_data)


In [374]:
train,test = split(index_list,ngrams,train_percentage,train_pid_remove,test_pid_remove)   
print(train.shape,test.shape)


(63, 2, 6) (7, 2, 7)
